In [59]:
import pandas as pd
import plotly 
import numpy as np
import plotly.figure_factory as ff
import scipy
#pip install scipy

In [60]:
# Import data

data = pd.read_csv('chocolate.csv')
continent = pd.read_csv('countryContinent.csv',encoding = "ISO-8859-1")
imp_exp=pd.read_csv('UNdata_Export_20220301_151116452.csv')

In [61]:
data.head()

,Unnamed: 0,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,...,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,first_taste,second_taste,third_taste,fourth_taste
0,0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,blackberry,full body,NaN
1,1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,vegetal,savory,NaN
2,2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,rich cocoa,fatty,bready,NaN
3,3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,...,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,fruity,melon,roasty,NaN
4,4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,...,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,vegetal,nutty,NaN,NaN


In [62]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [63]:
imp_exp=imp_exp[imp_exp['Commodity']=='Cocoa beans, whole or broken, raw or roasted'][['Country or Area','Year','Commodity','Flow','Quantity','Trade (USD)']]
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [64]:
# Prepar the data for the merge
data["company_location"] = data["company_location"].str.title()

data["company_location"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)'}, inplace=True)

In [65]:
# removing Unnamed:0
data=data.iloc[:,1:]

In [66]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [67]:
teste=data.merge(continent[['country','continent','sub_region']].rename(columns={'continent':'company_continent','sub_region':'company_region'}), left_on='company_location', right_on='country', how='left')

In [68]:
teste[teste['company_continent'].isna()]['company_location'].value_counts

<bound method IndexOpsMixin.value_counts of 193        Wales
486     Scotland
487     Scotland
488     Scotland
489     Scotland
490     Scotland
491     Scotland
1083    Scotland
1084    Scotland
1513    Scotland
1514    Scotland
Name: company_location, dtype: object>

## Excluimos Scotland visto que faz parte do UK?

In [69]:
teste=teste.merge(continent[['country','continent','sub_region']].rename(columns={'continent':'bean_continent','sub_region':'bean_region'}), left_on='country_of_bean_origin', right_on='country', how='left')

In [70]:
teste.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,first_taste,second_taste,third_taste,fourth_taste,country_x,company_continent,company_region,country_y,bean_continent,bean_region
0,2454,5150,United States of America,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,...,cocoa,blackberry,full body,NaN,United States of America,Americas,Northern America,Madagascar,Africa,Eastern Africa
1,2458,5150,United States of America,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,...,cocoa,vegetal,savory,NaN,United States of America,Americas,Northern America,NaN,NaN,NaN
2,2454,5150,United States of America,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,...,rich cocoa,fatty,bready,NaN,United States of America,Americas,Northern America,NaN,NaN,NaN
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,...,fruity,melon,roasty,NaN,France,Europe,Western Europe,Peru,Americas,South America
4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,have_bean,...,vegetal,nutty,NaN,NaN,France,Europe,Western Europe,NaN,NaN,NaN


In [71]:
data=teste

In [89]:
var1_2=np.asarray(data[data['rating']<=2]['cocoa_percent'],dtype=np.float32)
var3=np.asarray(data[data['rating']==3]['cocoa_percent'],dtype=np.float32)
var4=np.asarray(data[data['rating']==4]['cocoa_percent'],dtype=np.float32)

In [91]:
# falta meter titulo xaxis yaxis
labels = ['Rating 1 and 2','Rating 3','Rating 4']
variables=[var1_2,var3,var4]
fig = ff.create_distplot(variables,labels, show_hist=False)
fig.show()
